# Übungsaufgaben

In [ ]:
import networkx as nx
from pyspark import SparkContext

from tui_dsmt.parallel import MapReduce, Pregel
from tui_dsmt.parallel.datasets import load_texts, load_pregel_pagerank, text_paths

## Aufgaben 1: MapReduce
Verwenden Sie MapReduce, um die **Häufigkeit der einzelnen Buchstaben** in einer Menge von Texten zu zählen. Ignorieren Sie dabei Sonder-, Steuer- und Satzzeichen. Zählen Sie Groß- und Kleinbuchstaben zusammen.

Wir verwenden erneut einige deutsche Klassiker als Beispieltexte:

In [ ]:
texts = load_texts()

mr = MapReduce(texts, num_nodes=4)
mr.show()

Schreiben Sie zuerst die Map-Funktion.

In [ ]:
def map_function(title, data):
    # yield key, value
    pass

In [ ]:
map_result = mr.map(map_function)
map_result.show()

Implementieren Sie eine Combine-Funktion. Zur Erinnerung: Die Combine-Funktion soll die Daten lokal vor dem Shuffle-Schritt reduzieren, um die Menge der zwischen den Knoten versendeten Daten zu verringern.

In [ ]:
def combine_function(key, local_values):
    # yield key, combined_value
    pass

In [ ]:
combine_result = map_result.combine(combine_function)
combine_result.show()

Die Shuffle-Phase wird durch das Framework durchgeführt:

In [ ]:
shuffle_result = combine_result.shuffle()
shuffle_result.show(print_number_of_messages=True)

Implementieren Sie zuletzt eine Reduce-Funktion, um das gewünschte Ergebnis zu erhalten:

In [ ]:
def reduce_function(key, values):
    # yield key, reduced_value
    pass

In [ ]:
reduce_result = shuffle_result.reduce(reduce_function)
reduce_result.show()

## Aufgabe 2: Pregel
Implementieren Sie den PageRank-Algorithmus mit Pregel. Zur Erinnerung:

> Zur Berechnung der PageRank-Wertung wurde ursprünglich eine rekursive Formel aufgestellt. Benötigt wird zur Berechnung des Wertes für einen Knoten $n$ die Größe des Netzwerks $N$, die Menge der Nachbarn $S_n$, auf die $n$ verweisen, und ein Dämpfungsfaktor $d \in (0, 1)$, der üblicherweise mit $0.85$ angenommen wird.
>
>$$
C_{PR}(n) = \frac{1 - d}{N} + d * \sum_{m \in S_n} \frac{C_{PR}(m)}{deg(m)}
$$
>
> Die Initialisierung der Knoten wird in der Regel für jeden Knoten mit $\frac{1.0}{N}$ durchgeführt.

Ausgeführt werden soll der Algorithmus auf dem folgenden Graphen:

In [ ]:
graph, pos = load_pregel_pagerank()
nx.draw(graph, pos, with_labels=True, font_color='whitesmoke')

In unserem vereinfachten Modell können Sie dabei die Größe des Netzwerks und den Dämpfungsfaktor als global vorhandene Variablen annehmen.

In [ ]:
N = len(graph)
d = 0.85

N, d

Initialisieren Sie zunächst die Knoten mit dem korrekten Startwert. Denken Sie daran, dass jeder Knoten auch seinen eigenen **ausgehenden** Knotengrad weitergeben muss. Verwenden Sie daher das Format `(PageRank, Grad)`.

In [ ]:
def node_init(node):
    return ...

Definieren Sie nun eine Pregel-Funktion, die den PageRank berechnet und

In [ ]:
def pregel_fun(current_value, received_from, received_values, local_storage):
    return current_value, False

In [ ]:
Pregel(graph, node_init, pregel_fun, pos, print_messages=False)

## Aufgabe 3: Spark
Wiederholen Sie Aufgabe 1 mit PySpark.

In [ ]:
with SparkContext('local', 'letter count') as sc:
    input_rdd = sc.textFile(','.join(text_paths))

    char_counts = ...

char_counts